# Carbon footprint calculation

This exercise uses the method described in [Obringer et al. (2021)](http://doi.org/10.1016/j.resconrec.2020.105389) to create a carbon footprint of individual internet use.

One of the main factors is the energy mix in the user's home country. The model assumes that this is indicative of the greenhouse gas emission caused by the processing of the relevant data on the user's device and its transport across the network.

The analysis is limited in two major ways. First the assumption above does not account for international data transfers, and second only a limited set of internet services is included. In the original work, the authors reported minimum and maximum values, but in this calculation only median values are used.



## Code

As always, the first step is to import the Python packages we need to use.

In [ ]:
import pandas as pd

## Energy mix

The dataset describes the energy in TWh used by different entities (countries or regions) by type of fuel. It is available for download from [Our World in Data](https://ourworldindata.org/energy-mix) and the categories are

* Biofuels
* Coal
* Gas
* Geo Biomass Other
* Hydro
* Nuclear
* Oil
* Solar
* Wind

In [ ]:
df_raw = pd.read_csv('../shared/energy-consumption-by-source-and-country.csv')
df_raw.columns = [c[0:c.find(' - TWh')] if c.find(' - TWh') > 0 else c for c in df_raw.columns]
df_raw.columns = [c[0:c.find(' Consumption')] if c.find(' Consumption') > 0 else c for c in df_raw.columns]

## Reference data

The data in the next cell are provided by [Obringer et al. (2021)](http://doi.org/10.1016/j.resconrec.2020.105389)

The units are

* Fuel coefficients: g of CO2eq/kWh
* Elecricity usage: kWh/GB
* App data: GB/hour

In [ ]:
fuel_coefficients = pd.Series(
    index = ['Biofuels','Coal','Gas','Geo Biomass Other','Hydro','Nuclear','Oil','Solar','Wind'],
    data = [230,820,490,38,24,12,675,48,11]
    )
app_data = pd.Series(
    index = ['Netflix','Hulu','Amazon Video','Youtube','Spotify','Skype','Zoom','Webex','FaceTime','Google Hangout','Google Duo','Facebook','Twitter','Instagram','Snapchat','TikTok','WhatsApp','WeChat','Online Gaming','Web Browsing'],
    data = [4.0,4.3,4.0,8.1,0.0,1.0,1.5,1.5,0.2,1.8,0.5,0.1,0.4,0.1,0.2,0.9,0.2,0.1,0.2,0.2]
)
data_centre_usage = 0.01
transmission_usage = 0.06

## Prepare the country profile

The next cell filters the energy mix dataset according to the country specified. This creates a second pandas dataframe that is used in the remainder of the calculations. 

The units are

* Raw data: TWh/year
* Proportion is dimensionless
* Footprint: kWh/GB
* Coefficient: g of CO2eq/GB

In [ ]:
country = 'France'
year = 2021
df = df_raw[df_raw['Entity'] == country]
df = df[df['Year'] == year]
df = df.iloc[:,3:].transpose()
df.columns = [country]
country_total_TWh = float(df.sum())
df['proportion'] = df[country].apply(lambda x: x/country_total_TWh)
df['fuel_coefficient'] = [fuel_coefficients[i] for i in df.index ]
df = df.assign(footprint = df['proportion']  * df['fuel_coefficient'])
coefficient = df['footprint'].sum() * (data_centre_usage + transmission_usage)

## Personal data

We now have a coefficient that represents the carbon footprint of 1GB of data use in the selected country and a set of data that shows how much data is used by a particular application in an hour. The remaining variable is the number of hours spent using those application.

In the code below, identify the applications that you use. Estimate the number of hours you spend using that application in a typical week and enter the number into the dictinary assignment in place of the zero.

For example, if you estimate that you use Facebook for 3.5 hours a week, chang the code so that the relevant line reads

    ...
    'Facebook': 3.5,
    ...

In [ ]:
app_hours = pd.Series({
    'Amazon Video': 0,
    'Facebook': 0,
    'FaceTime': 0,
    'Google Hangout': 0,
    'Google Duo': 0,
    'Hulu': 0,
    'Instagram': 0,
    'Netflix': 0,
    'Online Gaming': 0,
    'Skype': 0,
    'Snapchat': 0,
    'Spotify': 0,
    'TikTok': 0,
    'Twitter': 0,
    'Web Browsing': 0,
    'Webex': 0,
    'WeChat': 0,
    'WhatsApp': 0,
    'Youtube': 0,
    'Zoom': 0
})

## Footprint calculation

Finally, calculate your internet carbon footprint by running the code in the next cell. The result is the kg of CO<sub>2</sub> equivalent.

In [ ]:
footprint = 0
for i in app_hours.index:
    footprint += app_data[i] * app_hours[i] * coefficient / 1000
    
print('Your personal internet footprint is {:.4f}kg of CO2eq per week ({:.4f}kg of CO2eq per year)'.format(footprint, footprint * 52))
